In [10]:
# Install Plotly for interactive financial visualizations
pip install plotly


[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [11]:
# Import libraries for data manipulation, visualization, and Plotly styling
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')
import plotly.io as pio
pio.templates.default = "plotly_white"

In [12]:
# Load the financial dataset from the CSV file
data=pd.read_csv("./Financials.csv")

In [13]:
# Filter the dataset to only include the Government segment
data=data[data['Segment']=='Government']

In [14]:
# Further filter the data to only include sales in Canada
data=data[data['Country']=='Canada']



In [15]:
# Inspect all column names to understand the dataset structure
data.columns

Index(['Segment', 'Country', ' Product ', ' Discount Band ', ' Units Sold ',
       ' Manufacturing Price ', ' Sale Price ', ' Gross Sales ', ' Discounts ',
       '  Sales ', ' COGS ', ' Profit ', 'Date', 'Month Number',
       ' Month Name ', 'Year'],
      dtype='object')

In [16]:
# Check the distinct product names available after filtering
data[' Product '].unique()

array([' Carretera ', ' Paseo ', ' VTT ', ' Montana ', ' Velo ',
       ' Amarilla '], dtype=object)

In [17]:
# Convert Sales from string currency format to numeric values
data['  Sales ']=data['  Sales '].str.replace('$','').str.replace(',','').astype(float)

In [18]:
# Convert Profit from string currency format to numeric values
data[' Profit ']=data[' Profit '].str.replace('$','').str.replace(',','').astype(float)

In [19]:
# Aggregate total sales for each product to see product performance
sales_by_product=data.groupby(' Product ')['  Sales '].sum()

In [20]:
# Display the aggregated sales by product as a clean table
sales_by_product.reset_index()

,Product,Sales
0,Amarilla,2579791.79
1,Carretera,1332156.65
2,Montana,829859.91
3,Paseo,3956873.45
4,VTT,667982.25
5,Velo,1374572.47


In [21]:
# Calculate total yearly profit to compare performance across years
Profit=data.groupby('Year')[' Profit '].sum()

In [22]:
# View the yearly profit totals we just computed
Profit

Year
2013     606726.12
2014    1651745.40
Name:  Profit , dtype: float64

In [23]:
# Convert Gross Sales from string currency format to numeric values
data[' Gross Sales ']=data[' Gross Sales '].str.replace('$','').str.replace(',','').astype(float)

In [24]:
# Sum gross sales by year to understand overall revenue trends
gross_profit=data.groupby('Year')[' Gross Sales '].sum()

In [25]:
# Display the yearly gross sales totals to inspect revenue levels
gross_profit

Year
2013    2589198.0
2014    9056159.0
Name:  Gross Sales , dtype: float64

In [26]:
# Clean and convert Discounts from string currency format to numeric values
data[' Discounts ']=data[' Discounts '].str.replace('$','').str.replace(',','').str.replace('-','').str.replace('    ','0').astype(float)

In [27]:
# Aggregate yearly discounts to understand total services or promotional costs
services=data.groupby('Year')[' Discounts '].sum()
services

Year
2013    110771.88
2014    793348.60
Name:  Discounts , dtype: float64

In [28]:
# Convert Cost of Goods Sold (COGS) from string currency format to numeric values
data[' COGS ']=data[' COGS '].str.replace('$','').str.replace(',','').astype(float)

In [29]:
# Sum yearly COGS to compute the total cost of revenue
cost_of_revenue=data.groupby('Year')[' COGS '].sum()

In [30]:
# Derive product cost by subtracting services cost (discounts) from total COGS
product_cost=cost_of_revenue.subtract(services)



In [31]:
# Re-display yearly services (discounts) to confirm the values used in product cost
services

Year
2013    110771.88
2014    793348.60
Name:  Discounts , dtype: float64

In [32]:
# Compute total revenue as gross sales plus derived product cost
revenue=gross_profit + product_cost

In [33]:
# Define simple assumptions for expenses and derive operating and net profit
r_and_d = 1200
sg_a = 1500
taxes = 1800
operating_expences=(r_and_d + sg_a)
operating_profit = gross_profit - (r_and_d + sg_a)
net_profit = operating_profit - taxes

In [34]:
# Define labels for all nodes used in the Sankey diagram
labels = [
    "Carretera", "Paseo", "VTT", "Montana", "Velo", "Amarilla","Products",
    "Revenue", "Cost of revenue","services cost","Products Cost", "Gross Profit","Operateing Expenses","R&D", "SG&A", "Taxes","Operating Profit", "Net Profit"
]

In [35]:
# Build the source, target, and value arrays that define flows for the Sankey diagram
source = [0, 1, 2, 3, 4, 5, 6, 6, 7, 8, 8, 7, 11, 12, 12, 16, 11, 16]
target = [6, 6, 6, 6, 6, 6, 7, None, 8, 9, 10, 11, 12, 12, 12, 12, 16, None]
value = [sales_by_product[0],sales_by_product[1],sales_by_product[2],sales_by_product[3],sales_by_product[4],sales_by_product[5],revenue[2014],revenue[2014],cost_of_revenue[2014],services[2014],product_cost[2014],gross_profit[2014],operating_expences,r_and_d,sg_a,taxes,operating_profit,net_profit[2014]]

In [38]:
# Upgrade nbformat to avoid potential notebook compatibility issues
!pip install --upgrade nbformat

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 675.7 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 1.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 822.9 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.5/390.5 kB 1.1 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [42]:
# Create and render the Sankey diagram that visualizes product-to-profit money flows
import plotly.graph_objects as go
import plotly.io as pio
fig = go.Figure(go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black"),
        label=labels # Add labels to nodes
    ),
    link=dict(
        source=source,
        target=target,
        value=value
    )
))
fig.update_layout(title_text="Canada Stock", font_size=10)
pio.renderers.default = "browser"
fig.show()


Opening in existing browser session.
